In [1]:
import pandas as pd
import requests
from xml.etree import ElementTree
import numpy as np
import folium
from folium.plugins import MarkerCluster
MarkerCluster()

In [ ]:
df_lookup = pd.read_csv(lookup_file)


In [3]:
GetAllCountersUrl = "http://webservices.commuterpage.com/counters.cfc?wsdl&method=GetAllCounters"

In [4]:
xmlfile = open('xml_getallcounters.xml', 'w')
xmldata = requests.get(GetAllCountersUrl)
xmlfile.write(xmldata.text)
xmlfile.close()

xml_data = 'xml_getallcounters.xml'

In [5]:
tree = ElementTree.parse(xml_data)
counter = tree.find('counter')
name = counter.find('name')
name.text

'110 Trail'

In [6]:
counter.find('latitude').text


'38.885315'

In [7]:
counter.find('longitude').text


'-77.065022'

In [8]:
counter.find('region/name').text


'Arlington'

In [9]:
id = []
name = []
latitude = []
longitude = []
region = []


for c in tree.findall('counter'):
    id.append(c.attrib['id'])
    name.append(c.find('name').text)
    latitude.append(c.find('latitude').text)
    longitude.append(c.find('longitude').text)
    region.append(c.find('region/name').text)

df_counters = pd.DataFrame(
    {'ID' : id,
     'Name' : name,
     'latitude' : latitude,
     'longitude' : longitude,
     'region' : region
    })
df_counters.head()

,ID,Name,latitude,longitude,region
0,33,110 Trail,38.885315,-77.065022,Arlington
1,30,14th Street Bridge,38.874260,-77.044610,Arlington
2,43,15th Street NW,38.907470,-77.034610,DC
3,32,Arlington Mill Trail,38.845610,-77.096046,Arlington
4,24,Ballston Connector,38.882950,-77.121235,Arlington


In [10]:
locations = df_counters[['latitude', 'longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

['38.979500', '-77.096760']

In [28]:
map = folium.Map(location=[38.9, -77.05], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(map)
map

In [22]:
def regioncolors(counter):
    if counter['region'] == 'Arlington':
        return 'green'
    elif counter['region'] == 'Alexandria':
        return 'blue'
    elif counter['region'] == 'DC':
        return 'red'
    else:
        return 'darkblue'
df_counters["color"] = df_counters.apply(regioncolors, axis=1)
df_counters.head()

,ID,Name,latitude,longitude,region,color
0,33,110 Trail,38.885315,-77.065022,Arlington,green
1,30,14th Street Bridge,38.874260,-77.044610,Arlington,green
2,43,15th Street NW,38.907470,-77.034610,DC,red
3,32,Arlington Mill Trail,38.845610,-77.096046,Arlington,green
4,24,Ballston Connector,38.882950,-77.121235,Arlington,green


In [27]:
map3 = folium.Map(location=[38.9, -77.05], tiles='CartoDB positron', zoom_start=11)

marker_cluster = MarkerCluster().add_to(map3)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup='ID:'+df_counters['ID'][point]+' '+df_counters['Name'][point], icon=folium.Icon(color=df_counters["color"][point], icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map3